In [1]:
from pathlib import Path
from datetime import date
import pandas as pd
import numpy as np

# Model-wide config (equivalent to config.py)
MODEL_VERSION = "V28"
PAYMENT_YEAR = 2026

DATE_ASOF = date(2026, 2, 1)
DATE_ASOF_EDIT = DATE_ASOF
ID_COL = "MBI"

# TODO: paste full lists from SAS macros V2825T1M:


DEM_VARS = [
    "AGEF", "ORIGDS", "DISABL",
    "F0_34","F35_44","F45_54","F55_59","F60_64","F65_69",
    "F70_74","F75_79","F80_84","F85_89","F90_94","F95_GT",
    "M0_34","M35_44","M45_54","M55_59","M60_64","M65_69",
    "M70_74","M75_79","M80_84","M85_89","M90_94","M95_GT",
    "NEF0_34","NEF35_44","NEF45_54","NEF55_59","NEF60_64",
    "NEF65","NEF66","NEF67","NEF68","NEF69",
    "NEF70_74","NEF75_79","NEF80_84","NEF85_89","NEF90_94","NEF95_GT",
    "NEM0_34","NEM35_44","NEM45_54","NEM55_59","NEM60_64",
    "NEM65","NEM66","NEM67","NEM68","NEM69",
    "NEM70_74","NEM75_79","NEM80_84","NEM85_89","NEM90_94","NEM95_GT",
]


In [9]:
HCC_LIST = [
    "HCC1","HCC2","HCC6","HCC17","HCC18","HCC19","HCC20","HCC21","HCC22","HCC23",
    "HCC35","HCC36","HCC37","HCC38","HCC48","HCC49","HCC50","HCC51","HCC62","HCC63",
    "HCC64","HCC65","HCC68","HCC77","HCC78","HCC79","HCC80","HCC81","HCC92","HCC93",
    "HCC94","HCC107","HCC108","HCC109","HCC111","HCC112","HCC114","HCC115","HCC125",
    "HCC126","HCC127","HCC135","HCC136","HCC137","HCC138","HCC139","HCC151","HCC152",
    "HCC153","HCC154","HCC155","HCC180","HCC181","HCC182","HCC190","HCC191","HCC192",
    "HCC193","HCC195","HCC196","HCC197","HCC198","HCC199","HCC200","HCC201","HCC202",
    "HCC211","HCC212","HCC213","HCC221","HCC222","HCC223","HCC224","HCC225","HCC226",
    "HCC227","HCC228","HCC229","HCC238","HCC248","HCC249","HCC253","HCC254","HCC263",
    "HCC264","HCC267","HCC276","HCC277","HCC278","HCC279","HCC280","HCC282","HCC283",
    "HCC298","HCC300","HCC326","HCC327","HCC328","HCC329","HCC379","HCC380","HCC381",
    "HCC382","HCC383","HCC385","HCC387","HCC397","HCC398","HCC399","HCC401","HCC402",
    "HCC405","HCC409","HCC454","HCC463",
]

CC_LIST = [
    "CC1","CC2","CC6","CC17","CC18","CC19","CC20","CC21","CC22","CC23",
    "CC35","CC36","CC37","CC38","CC48","CC49","CC50","CC51","CC62","CC63",
    "CC64","CC65","CC68","CC77","CC78","CC79","CC80","CC81","CC92","CC93",
    "CC94","CC107","CC108","CC109","CC111","CC112","CC114","CC115","CC125",
    "CC126","CC127","CC135","CC136","CC137","CC138","CC139","CC151","CC152",
    "CC153","CC154","CC155","CC180","CC181","CC182","CC190","CC191","CC192",
    "CC193","CC195","CC196","CC197","CC198","CC199","CC200","CC201","CC202",
    "CC211","CC212","CC213","CC221","CC222","CC223","CC224","CC225","CC226",
    "CC227","CC228","CC229","CC238","CC248","CC249","CC253","CC254","CC263",
    "CC264","CC267","CC276","CC277","CC278","CC279","CC280","CC282","CC283",
    "CC298","CC300","CC326","CC327","CC328","CC329","CC379","CC380","CC381",
    "CC382","CC383","CC385","CC387","CC397","CC398","CC399","CC401","CC402",
    "CC405","CC409","CC454","CC463",
]


In [23]:
# Configure input/output paths relative to notebook
BASE = Path.cwd().parent  # notebook in notebooks/; adjust if needed

person_path = BASE / "data" / "person.csv"
diag_path   = BASE / "data" / "diag.csv"
icd_cc_path = BASE / "data" / "reference" / "2026 Initial ICD-10-CM Mappings.csv"
coef_xlsx   = BASE / "data" / "reference" / "C2824T2N_new.xlsx"

output_dir  = BASE / "data" / "output"
output_dir.mkdir(parents=True, exist_ok=True)
output_path = output_dir / "v28_hcc_person_scores.csv"

def load_person_csv(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path, dtype={"MBI": str, "SEX": str, "OREC": str})
    df["DOB"] = pd.to_datetime(df["DOB"])
    return df

def load_diag_csv(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path, dtype={"MBI": str, "DIAG": str})
    df["DIAG"] = df["DIAG"].str.strip().str.upper()
    return df

# def load_icd10_cc_map(path: Path) -> pd.DataFrame:
#     df = pd.read_csv(path, dtype=str)
#     df["ICD10"] = df["ICD10"].str.strip().str.upper()
#     df["CC"] = df["CC"].astype(str).str.strip()
#     return df
def load_icd10_cc_map(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path, dtype=str)

    # Normalize column names: strip spaces, replace newlines with spaces, collapse multiple spaces
    new_cols = []
    for c in df.columns:
        c_norm = c.replace("\n", " ").strip()
        while "  " in c_norm:
            c_norm = c_norm.replace("  ", " ")
        new_cols.append(c_norm)
    df.columns = new_cols

    print("Normalized columns:", df.columns.tolist())

    # Now rename the two columns we care about
    df = df.rename(columns={
        "Diagnosis Code": "ICD10",
        "CMS-HCC Model Category V28 for 2025 Payment Year": "CC",
    })

    if "ICD10" not in df.columns or "CC" not in df.columns:
        raise ValueError("ICD10 or CC column not found after normalization/rename")

    df["ICD10"] = df["ICD10"].str.strip().str.upper()
    df["CC"] = df["CC"].astype(str).str.strip()

    # Keep only rows with a V28 CC
    df = df[df["CC"].notna() & (df["CC"] != "")]
    return df

#what is wrong here?

# Coefficient prefix mapping – adjust if your sheet uses different prefixes
PREFIX_TO_MODEL = {
    "CNA_": "COMMUNITY_NA",          # Community Non-dual Aged
    "CND_": "COMMUNITY_ND",          # Community Non-dual Disabled
    "CFBA_": "COMMUNITY_FBA",
    "CFBD_": "COMMUNITY_FBD",
    "CPBA_": "COMMUNITY_PBA",
    "CPBD_": "COMMUNITY_PBD",
    "INST_": "INSTITUTIONAL",
    "NE_":   "NEW_ENROLLEE",
    "SNE_":  "SNP_NEW_ENROLLEE",
}

def load_coefficients_xlsx(path: Path) -> pd.DataFrame:
    df_raw = pd.read_excel(path)
    cols = {c: c.strip().lower() for c in df_raw.columns}
    df_raw = df_raw.rename(columns=cols)
    if "name" not in df_raw.columns or "coeff" not in df_raw.columns:
        raise ValueError("Expected 'Name' and 'Coeff' columns in coefficients file.")
    records = []
    for _, row in df_raw.iterrows():
        name = str(row["name"])
        coef = float(row["coeff"])
        model = None
        for prefix, model_id in PREFIX_TO_MODEL.items():
            if name.startswith(prefix):
                model = model_id
                var_name = name[len(prefix):]
                break
        if model is None:
            continue
        records.append({"model": model, "var_name": var_name, "coef": coef})
    return pd.DataFrame.from_records(records)

person_df = load_person_csv(person_path)
diag_df   = load_diag_csv(diag_path)
icd_cc_df = load_icd10_cc_map(icd_cc_path)
coef_df   = load_coefficients_xlsx(coef_xlsx)

person_df.head(), diag_df.head(), icd_cc_df.head(), coef_df.head()


Normalized columns: ['Diagnosis Code', 'Description', 'CMS-HCC ESRD Model Category V21', 'CMS-HCC ESRD Model Category V24', 'CMS-HCC Model Category V22', 'CMS-HCC Model Category V24', 'CMS-HCC Model Category V28', 'RxHCC Model Category V08', 'CMS-HCC ESRD Model Category V21 for 2025 Payment Year', 'CMS-HCC ESRD Model Category V24 for 2025 Payment Year', 'CMS-HCC Model Category V22 for 2025 Payment Year', 'CMS-HCC Model Category V24 for 2025 Payment Year', 'CMS-HCC Model Category V28 for 2025 Payment Year', 'RxHCC Model Category V08 for 2025 Payment Year']


(    MBI SEX        DOB  LTIMCAID  NEMCAID OREC
 0  P001   1 1950-06-15         1        0    0
 1  P002   2 1980-03-10         0        1    1
 2  P003   1 1965-12-01         0        0    0,
     MBI   DIAG
 0  P001   E119
 1  P001   I509
 2  P002  C3490
 3  P003   J449,
    ICD10            Description CMS-HCC ESRD Model Category V21  \
 0  A0103      Typhoid pneumonia                             115   
 1  A0104      Typhoid arthritis                              39   
 2  A0105  Typhoid osteomyelitis                              39   
 3   A021      Salmonella sepsis                               2   
 4  A0222   Salmonella pneumonia                             115   
 
   CMS-HCC ESRD Model Category V24 CMS-HCC Model Category V22  \
 0                             115                        115   
 1                              39                         39   
 2                              39                         39   
 3                               2                      

In [22]:
pip install openpyxl


   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   ---------------------------------------- 2/2 [openpyxl]

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\avyay\py313_env\Scripts\python.exe -m pip install --upgrade pip


In [29]:

#changes from here

def compute_age_vars(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    dob = pd.to_datetime(df["DOB"])
    ref = pd.to_datetime(DATE_ASOF)
    months = (ref.year - dob.dt.year) * 12 + (ref.month - dob.dt.month)
    months -= (ref.day < dob.dt.day).astype(int)
    df["AGEF"] = (months // 12).clip(lower=0)

    ref_edit = pd.to_datetime(DATE_ASOF_EDIT)
    months_edit = (ref_edit.year - dob.dt.year) * 12 + (ref_edit.month - dob.dt.month)
    months_edit -= (ref_edit.day < dob.dt.day).astype(int)
    df["AGEF_EDIT"] = (months_edit // 12).clip(lower=0)
    return df

def add_age_sex_dummies(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    age = df["AGEF"]
    sex = df["SEX"].astype(str)
    orec = df["OREC"].astype(str)

    df["DISABL"] = ((age < 65) & (orec != "0")).astype(int)
    df["ORIGDS"] = ((orec == "1") & (df["DISABL"] == 0)).astype(int)

    cols_24 = [
        "F0_34","F35_44","F45_54","F55_59","F60_64","F65_69",
        "F70_74","F75_79","F80_84","F85_89","F90_94","F95_GT",
        "M0_34","M35_44","M45_54","M55_59","M60_64","M65_69",
        "M70_74","M75_79","M80_84","M85_89","M90_94","M95_GT",
    ]
    for c in cols_24:
        df[c] = 0

    def agesex_index(row):
        a = row["AGEF"]
        s = row["SEX"]
        if s == "2":
            if 0 <= a <= 34: return 1
            if 34 < a <= 44: return 2
            if 44 < a <= 54: return 3
            if 54 < a <= 59: return 4
            if 59 < a <= 64: return 5
            if 64 < a <= 69: return 6
            if 69 < a <= 74: return 7
            if 74 < a <= 79: return 8
            if 79 < a <= 84: return 9
            if 84 < a <= 89: return 10
            if 89 < a <= 94: return 11
            if a > 94:       return 12
        if s == "1":
            if 0 <= a <= 34: return 13
            if 34 < a <= 44: return 14
            if 44 < a <= 54: return 15
            if 54 < a <= 59: return 16
            if 59 < a <= 64: return 17
            if 64 < a <= 69: return 18
            if 69 < a <= 74: return 19
            if 74 < a <= 79: return 20
            if 79 < a <= 84: return 21
            if 84 < a <= 89: return 22
            if 89 < a <= 94: return 23
            if a > 94:       return 24
        return np.nan

    idx = df.apply(agesex_index, axis=1)
    for i, col in enumerate(cols_24, start=1):
        df.loc[idx == i, col] = 1

    # NEF*/NEM* – implement NE_AGESEX logic from AGESEXV2.TXT here (straight copy of CASE rules). [file:35]
    # For now, initialize to 0 so code runs; you can fill exact rules later.
#     ne_cols = [
#         "NEF0_34","NEF35_44","NEF45_54","NEF55_59","NEF60_64",
#         "NEF65","NEF66","NEF67","NEF68","NEF69",
#         "NEF70_74","NEF75_79","NEF80_84","NEF85_89","NEF90_94","NEF95_GT",
#         "NEM0_34","NEM35_44","NEM45_54","NEM55_59","NEM60_64",
#         "NEM65","NEM66","NEM67","NEM68","NEM69",
#         "NEM70_74","NEM75_79","NEM80_84","NEM85_89","NEM90_94","NEM95_GT",
#     ]
#     for c in ne_cols:
#         df[c] = 0

#     return df
        # --- New Enrollee age/sex dummies (NEF*/NEM*) --- [file:35]
    ne_cols = [
        "NEF0_34","NEF35_44","NEF45_54","NEF55_59","NEF60_64",
        "NEF65","NEF66","NEF67","NEF68","NEF69",
        "NEF70_74","NEF75_79","NEF80_84","NEF85_89","NEF90_94","NEF95_GT",
        "NEM0_34","NEM35_44","NEM45_54","NEM55_59","NEM60_64",
        "NEM65","NEM66","NEM67","NEM68","NEM69",
        "NEM70_74","NEM75_79","NEM80_84","NEM85_89","NEM90_94","NEM95_GT",
    ]
    for c in ne_cols:
        df[c] = 0

    def ne_agesex_index(row):
        a = row["AGEF"]
        s = row["SEX"]
        o = row["OREC"]
        # Female (SEX='2') [file:35]
        if s == "2":
            if 0 <= a <= 34: return 1   # NEF0_34
            if 34 < a <= 44: return 2   # NEF35_44
            if 44 < a <= 54: return 3   # NEF45_54
            if 54 < a <= 59: return 4   # NEF55_59
            if 59 < a <= 63: return 5   # NEF60_64
            if a == 64 and o != "0": return 5   # disabled 64 -> 60-64 bucket
            if a == 64 and o == "0": return 6   # old-age 64 -> 65 bucket
            if a == 65: return 6       # NEF65
            if a == 66: return 7       # NEF66
            if a == 67: return 8       # NEF67
            if a == 68: return 9       # NEF68
            if a == 69: return 10      # NEF69
            if 69 < a <= 74: return 11 # NEF70_74
            if 74 < a <= 79: return 12 # NEF75_79
            if 79 < a <= 84: return 13 # NEF80_84
            if 84 < a <= 89: return 14 # NEF85_89
            if 89 < a <= 94: return 15 # NEF90_94
            if a > 94:       return 16 # NEF95_GT
        # Male (SEX='1') [file:35]
        if s == "1":
            if 0 <= a <= 34: return 17  # NEM0_34
            if 34 < a <= 44: return 18  # NEM35_44
            if 44 < a <= 54: return 19  # NEM45_54
            if 54 < a <= 59: return 20  # NEM55_59
            if 59 < a <= 63: return 21  # NEM60_64
            if a == 64 and o != "0": return 21  # disabled 64 -> 60-64 bucket
            if a == 64 and o == "0":  return 22 # old-age 64 -> 65 bucket
            if a == 65: return 22     # NEM65
            if a == 66: return 23     # NEM66
            if a == 67: return 24     # NEM67
            if a == 68: return 25     # NEM68
            if a == 69: return 26     # NEM69
            if 69 < a <= 74: return 27 # NEM70_74
            if 74 < a <= 79: return 28 # NEM75_79
            if 79 < a <= 84: return 29 # NEM80_84
            if 84 < a <= 89: return 30 # NEM85_89
            if 89 < a <= 94: return 31 # NEM90_94
            if a > 94:       return 32 # NEM95_GT
        return np.nan

    ne_idx = df.apply(ne_agesex_index, axis=1)

    # Map index 1..16 to NEF*, 17..32 to NEM* exactly like NECELL array in SAS [file:35]
    ne_order = [
        "NEF0_34","NEF35_44","NEF45_54","NEF55_59","NEF60_64",
        "NEF65","NEF66","NEF67","NEF68","NEF69",
        "NEF70_74","NEF75_79","NEF80_84","NEF85_89","NEF90_94","NEF95_GT",
        "NEM0_34","NEM35_44","NEM45_54","NEM55_59","NEM60_64",
        "NEM65","NEM66","NEM67","NEM68","NEM69",
        "NEM70_74","NEM75_79","NEM80_84","NEM85_89","NEM90_94","NEM95_GT",
    ]
    for i, col in enumerate(ne_order, start=1):
        df.loc[ne_idx == i, col] = 1

    return df



In [30]:
def build_cc_flags(person_df, diag_df, icd_cc_df) -> pd.DataFrame:
    d = diag_df.merge(icd_cc_df, left_on="DIAG", right_on="ICD10", how="left")
    d = d[d["CC"].notna()]
    d["flag"] = 1
    cc_wide = (
        d.pivot_table(
            index=ID_COL,
            columns="CC",
            values="flag",
            aggfunc="max",
            fill_value=0,
        )
        .reset_index()
    )
    cc_wide.columns.name = None

    # rename numeric CC codes (e.g. "92") to "CC92"
    rename = {}
    for col in cc_wide.columns:
        if col != ID_COL and not col.startswith("CC"):
            rename[col] = f"CC{col}"
    cc_wide = cc_wide.rename(columns=rename)

    for cc_var in CC_LIST:
        if cc_var not in cc_wide.columns:
            cc_wide[cc_var] = 0

    return cc_wide

# def apply_hierarchy(person_df, cc_flags_df) -> pd.DataFrame:
#     df = person_df.merge(cc_flags_df, on=ID_COL, how="left").fillna(0)

#     # Copy CC -> HCC assuming HCC_LIST and CC_LIST aligned by index [file:36][file:31]
#     for cc_var, hcc_var in zip(CC_LIST, HCC_LIST):
#         df[hcc_var] = df[cc_var]

#     def set0(df, cc_num, hier_nums):
#         cc_col = f"HCC{cc_num}"
#         if cc_col not in df.columns:
#             return df
#         mask = df[cc_col] == 1
#         for h in hier_nums:
#             h_col = f"HCC{h}"
#             if h_col in df.columns:
#                 df.loc[mask, h_col] = 0
#         return df

#     # Paste all SET0 lines from V28115H1.TXT here. Example: [file:36]
#     df = set0(df, 17, [18, 19, 20, 21, 22, 23])
#     df = set0(df, 18, [19, 20, 21, 22, 23])
#     df = set0(df, 19, [20, 21, 22, 23])
#     df = set0(df, 20, [21, 22, 23])
#     df = set0(df, 21, [22, 23])
#     df = set0(df, 22, [23])
#     # ...continue for all Diabetes, Liver, GI, Heart, Lung, etc hierarchies in V28115H1.TXT

#     return df
def apply_hierarchy(person_df, cc_flags_df) -> pd.DataFrame:
    df = person_df.merge(cc_flags_df, on=ID_COL, how="left").fillna(0)

    # Copy CC -> HCC assuming HCC_LIST and CC_LIST aligned by index [file:36][file:38]
    for cc_var, hcc_var in zip(CC_LIST, HCC_LIST):
        df[hcc_var] = df[cc_var]

    def set0(df, cc_num, hier_nums):
        cc_col = f"HCC{cc_num}"
        if cc_col not in df.columns:
            return df
        mask = df[cc_col] == 1
        for h in hier_nums:
            h_col = f"HCC{h}"
            if h_col in df.columns:
                df.loc[mask, h_col] = 0
        return df

    # --- Hierarchies from V28115H1.TXT --- [file:36]

    # Neoplasm
    df = set0(df, 17, [18, 19, 20, 21, 22, 23])
    df = set0(df, 18, [19, 20, 21, 22, 23])
    df = set0(df, 19, [20, 21, 22, 23])
    df = set0(df, 20, [21, 22, 23])
    df = set0(df, 21, [22, 23])
    df = set0(df, 22, [23])

    # Diabetes
    df = set0(df, 35, [36, 37, 38])
    df = set0(df, 36, [37, 38])
    df = set0(df, 37, [38])

    # Liver
    df = set0(df, 62, [63, 64, 65, 68])
    df = set0(df, 63, [64, 65, 68, 202])
    df = set0(df, 64, [65, 68])

    # GI
    df = set0(df, 77, [78, 80, 81])
    df = set0(df, 80, [81])

    # MSK
    df = set0(df, 93, [94])

    # Blood
    df = set0(df, 107, [108])
    df = set0(df, 111, [112])
    df = set0(df, 114, [115])

    # Cognitive
    df = set0(df, 125, [126, 127])
    df = set0(df, 126, [127])

    # Substance Use Disorder (SUD)
    df = set0(df, 135, [136, 137, 138, 139])
    df = set0(df, 136, [137, 138, 139])
    df = set0(df, 137, [138, 139])
    df = set0(df, 138, [139])

    # Psychiatric
    df = set0(df, 151, [152, 153, 154, 155])
    df = set0(df, 152, [153, 154, 155])
    df = set0(df, 153, [154, 155])
    df = set0(df, 154, [155])

    # Spinal
    df = set0(df, 180, [181, 182, 253, 254])
    df = set0(df, 181, [182, 254])

    # Neuro
    df = set0(df, 191, [180, 181, 182, 192, 253, 254])
    df = set0(df, 192, [180, 181, 182, 253, 254])
    df = set0(df, 195, [196])

    # Arrest
    df = set0(df, 211, [212, 213])
    df = set0(df, 212, [213])

    # Heart
    df = set0(df, 221, [222, 223, 224, 225, 226, 227])
    df = set0(df, 222, [223, 224, 225, 226, 227])
    df = set0(df, 223, [224, 225, 226, 227])
    df = set0(df, 224, [225, 226, 227])
    df = set0(df, 225, [226, 227])
    df = set0(df, 226, [227])
    df = set0(df, 228, [229])

    # CVD
    df = set0(df, 248, [249])
    df = set0(df, 253, [254])

    # Vascular
    df = set0(df, 263, [264, 383, 409])

    # Lung
    df = set0(df, 276, [277, 278, 279, 280])
    df = set0(df, 277, [278, 279, 280])
    df = set0(df, 278, [279, 280])
    df = set0(df, 279, [280])
    df = set0(df, 282, [283])

    # Kidney
    df = set0(df, 326, [327, 328, 329])
    df = set0(df, 327, [328, 329])
    df = set0(df, 328, [329])

    # Skin
    df = set0(df, 379, [380, 381, 382, 383])
    df = set0(df, 380, [381, 382, 383])
    df = set0(df, 381, [382, 383])
    df = set0(df, 382, [383])

    # Injury
    df = set0(df, 397, [202, 398, 399])
    df = set0(df, 398, [202, 399])
    df = set0(df, 405, [409])

    return df



In [31]:
# TODO: paste these from V2825T1M.TXT [file:31]
COMM_REGA = [
    # &AGESEXVA
    "F65_69",
    "F70_74", "F75_79", "F80_84", "F85_89", "F90_94", "F95_GT",
    "M65_69",
    "M70_74", "M75_79", "M80_84", "M85_89", "M90_94", "M95_GT",
    # &ORIG_INT
    "OriginallyDisabled_Female", "OriginallyDisabled_Male",
    # &HCClist  (HCCV28_list115)
    *HCC_LIST,
    # &INTERRACC_VARSA
    "DIABETES_HF_V28",
    "HF_CHR_LUNG_V28",
    "HF_KIDNEY_V28",
    "CHR_LUNG_CARD_RESP_FAIL_V28",
    "HF_HCC238_V28",
    # &ADDZ
    "D1", "D2", "D3", "D4", "D5", "D6", "D7", "D8", "D9", "D10P",
]

COMM_REGD = [
    # &AGESEXVD
    "F0_34", "F35_44", "F45_54", "F55_59", "F60_64",
    "M0_34", "M35_44", "M45_54", "M55_59", "M60_64",
    # &HCClist
    *HCC_LIST,
    # &INTERRACC_VARSD
    "DIABETES_HF_V28",
    "HF_CHR_LUNG_V28",
    "HF_KIDNEY_V28",
    "CHR_LUNG_CARD_RESP_FAIL_V28",
    "HF_HCC238_V28",
    "gSubUseDisorder_gPsych_V28",
    # &ADDZ
    "D1", "D2", "D3", "D4", "D5", "D6", "D7", "D8", "D9", "D10P",
]

INST_REG = [
    # &AGESEXV
    "F0_34", "F35_44", "F45_54", "F55_59", "F60_64", "F65_69",
    "F70_74", "F75_79", "F80_84", "F85_89", "F90_94", "F95_GT",
    "M0_34", "M35_44", "M45_54", "M55_59", "M60_64", "M65_69",
    "M70_74", "M75_79", "M80_84", "M85_89", "M90_94", "M95_GT",
    # LTIMCAID ORIGDS
    "LTIMCAID", "ORIGDS",
    # &HCClist
    *HCC_LIST,
    # &INTERRACI_VARS
    "DIABETES_HF_V28",
    "HF_CHR_LUNG_V28",
    "HF_KIDNEY_V28",
    "CHR_LUNG_CARD_RESP_FAIL_V28",
    "DISABLED_CANCER_V28",
    "DISABLED_NEURO_V28",
    "DISABLED_HF_V28",
    "DISABLED_CHR_LUNG_V28",
    "DISABLED_ULCER_V28",
    # &ADDZ
    "D1", "D2", "D3", "D4", "D5", "D6", "D7", "D8", "D9", "D10P",
]

NE_REG = [
   "NMCAID_NORIGDIS_NEF0_34",  "NMCAID_NORIGDIS_NEF35_44",
   "NMCAID_NORIGDIS_NEF45_54", "NMCAID_NORIGDIS_NEF55_59",
   "NMCAID_NORIGDIS_NEF60_64", "NMCAID_NORIGDIS_NEF65",
   "NMCAID_NORIGDIS_NEF66",    "NMCAID_NORIGDIS_NEF67",
   "NMCAID_NORIGDIS_NEF68",    "NMCAID_NORIGDIS_NEF69",
   "NMCAID_NORIGDIS_NEF70_74", "NMCAID_NORIGDIS_NEF75_79",
   "NMCAID_NORIGDIS_NEF80_84", "NMCAID_NORIGDIS_NEF85_89",
   "NMCAID_NORIGDIS_NEF90_94", "NMCAID_NORIGDIS_NEF95_GT",

   "NMCAID_NORIGDIS_NEM0_34",  "NMCAID_NORIGDIS_NEM35_44",
   "NMCAID_NORIGDIS_NEM45_54", "NMCAID_NORIGDIS_NEM55_59",
   "NMCAID_NORIGDIS_NEM60_64", "NMCAID_NORIGDIS_NEM65",
   "NMCAID_NORIGDIS_NEM66",    "NMCAID_NORIGDIS_NEM67",
   "NMCAID_NORIGDIS_NEM68",    "NMCAID_NORIGDIS_NEM69",
   "NMCAID_NORIGDIS_NEM70_74", "NMCAID_NORIGDIS_NEM75_79",
   "NMCAID_NORIGDIS_NEM80_84", "NMCAID_NORIGDIS_NEM85_89",
   "NMCAID_NORIGDIS_NEM90_94", "NMCAID_NORIGDIS_NEM95_GT",

   "MCAID_NORIGDIS_NEF0_34",   "MCAID_NORIGDIS_NEF35_44",
   "MCAID_NORIGDIS_NEF45_54",  "MCAID_NORIGDIS_NEF55_59",
   "MCAID_NORIGDIS_NEF60_64",  "MCAID_NORIGDIS_NEF65",
   "MCAID_NORIGDIS_NEF66",     "MCAID_NORIGDIS_NEF67",
   "MCAID_NORIGDIS_NEF68",     "MCAID_NORIGDIS_NEF69",
   "MCAID_NORIGDIS_NEF70_74",  "MCAID_NORIGDIS_NEF75_79",
   "MCAID_NORIGDIS_NEF80_84",  "MCAID_NORIGDIS_NEF85_89",
   "MCAID_NORIGDIS_NEF90_94",  "MCAID_NORIGDIS_NEF95_GT",

   "MCAID_NORIGDIS_NEM0_34",   "MCAID_NORIGDIS_NEM35_44",
   "MCAID_NORIGDIS_NEM45_54",  "MCAID_NORIGDIS_NEM55_59",
   "MCAID_NORIGDIS_NEM60_64",  "MCAID_NORIGDIS_NEM65",
   "MCAID_NORIGDIS_NEM66",     "MCAID_NORIGDIS_NEM67",
   "MCAID_NORIGDIS_NEM68",     "MCAID_NORIGDIS_NEM69",
   "MCAID_NORIGDIS_NEM70_74",  "MCAID_NORIGDIS_NEM75_79",
   "MCAID_NORIGDIS_NEM80_84",  "MCAID_NORIGDIS_NEM85_89",
   "MCAID_NORIGDIS_NEM90_94",  "MCAID_NORIGDIS_NEM95_GT",

   "NMCAID_ORIGDIS_NEF65",     "NMCAID_ORIGDIS_NEF66",
   "NMCAID_ORIGDIS_NEF67",     "NMCAID_ORIGDIS_NEF68",
   "NMCAID_ORIGDIS_NEF69",     "NMCAID_ORIGDIS_NEF70_74",
   "NMCAID_ORIGDIS_NEF75_79",  "NMCAID_ORIGDIS_NEF80_84",
   "NMCAID_ORIGDIS_NEF85_89",  "NMCAID_ORIGDIS_NEF90_94",
   "NMCAID_ORIGDIS_NEF95_GT",

   "NMCAID_ORIGDIS_NEM65",     "NMCAID_ORIGDIS_NEM66",
   "NMCAID_ORIGDIS_NEM67",     "NMCAID_ORIGDIS_NEM68",
   "NMCAID_ORIGDIS_NEM69",     "NMCAID_ORIGDIS_NEM70_74",
   "NMCAID_ORIGDIS_NEM75_79",  "NMCAID_ORIGDIS_NEM80_84",
   "NMCAID_ORIGDIS_NEM85_89",  "NMCAID_ORIGDIS_NEM90_94",
   "NMCAID_ORIGDIS_NEM95_GT",

   "MCAID_ORIGDIS_NEF65",      "MCAID_ORIGDIS_NEF66",
   "MCAID_ORIGDIS_NEF67",      "MCAID_ORIGDIS_NEF68",
   "MCAID_ORIGDIS_NEF69",      "MCAID_ORIGDIS_NEF70_74",
   "MCAID_ORIGDIS_NEF75_79",   "MCAID_ORIGDIS_NEF80_84",
   "MCAID_ORIGDIS_NEF85_89",   "MCAID_ORIGDIS_NEF90_94",
   "MCAID_ORIGDIS_NEF95_GT",

   "MCAID_ORIGDIS_NEM65",      "MCAID_ORIGDIS_NEM66",
   "MCAID_ORIGDIS_NEM67",      "MCAID_ORIGDIS_NEM68",
   "MCAID_ORIGDIS_NEM69",      "MCAID_ORIGDIS_NEM70_74",
   "MCAID_ORIGDIS_NEM75_79",   "MCAID_ORIGDIS_NEM80_84",
   "MCAID_ORIGDIS_NEM85_89",   "MCAID_ORIGDIS_NEM90_94",
   "MCAID_ORIGDIS_NEM95_GT",
]


def score_linear(df: pd.DataFrame, var_list, coef_series) -> pd.Series:
    missing = [v for v in var_list if v not in df.columns]
    for v in missing:
        df[v] = 0.0
    aligned_coefs = coef_series.reindex(var_list).fillna(0.0)
    return (df[var_list] * aligned_coefs.values).sum(axis=1)

def compute_all_scores(df: pd.DataFrame, coef_df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    configs = [
        ("SCORE_COMMUNITY_NA",  "COMMUNITY_NA",  COMM_REGA),
        ("SCORE_COMMUNITY_ND",  "COMMUNITY_ND",  COMM_REGD),
        ("SCORE_COMMUNITY_FBA", "COMMUNITY_FBA", COMM_REGA),
        ("SCORE_COMMUNITY_FBD", "COMMUNITY_FBD", COMM_REGD),
        ("SCORE_COMMUNITY_PBA", "COMMUNITY_PBA", COMM_REGA),
        ("SCORE_COMMUNITY_PBD", "COMMUNITY_PBD", COMM_REGD),
        ("SCORE_INSTITUTIONAL", "INSTITUTIONAL", INST_REG),
        ("SCORE_NEW_ENROLLEE",  "NEW_ENROLLEE",  NE_REG),
        ("SCORE_SNP_NEW_ENROLLEE", "SNP_NEW_ENROLLEE", NE_REG),
    ]

    for out_var, model_name, var_list in configs:
        coef_series = (
            coef_df[coef_df["model"] == model_name]
            .set_index("var_name")["coef"]
        )
        df[out_var] = score_linear(df, var_list, coef_series)

    return df


In [32]:
# 1) AGE + demographics
person_with_age = compute_age_vars(person_df)
person_with_demo = add_age_sex_dummies(person_with_age)

# 2) CC flags from diagnoses
cc_flags_df = build_cc_flags(person_with_demo, diag_df, icd_cc_df)

# 3) Apply hierarchies to get HCCs
with_hcc = apply_hierarchy(person_with_demo, cc_flags_df)

# 4) Compute 9 scores
scored_df = compute_all_scores(with_hcc, coef_df)

print("Shape after scoring:", scored_df.shape)
scored_df.head()


Shape after scoring: (3, 437)


C:\Users\avyay\AppData\Local\Temp\ipykernel_1260\3706421954.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  cc_wide[cc_var] = 0
C:\Users\avyay\AppData\Local\Temp\ipykernel_1260\3706421954.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  cc_wide[cc_var] = 0
C:\Users\avyay\AppData\Local\Temp\ipykernel_1260\3706421954.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(a

,MBI,SEX,DOB,LTIMCAID,NEMCAID,OREC,AGEF,AGEF_EDIT,DISABL,ORIGDS,...,MCAID_ORIGDIS_NEM68,MCAID_ORIGDIS_NEM69,MCAID_ORIGDIS_NEM70_74,MCAID_ORIGDIS_NEM75_79,MCAID_ORIGDIS_NEM80_84,MCAID_ORIGDIS_NEM85_89,MCAID_ORIGDIS_NEM90_94,MCAID_ORIGDIS_NEM95_GT,SCORE_NEW_ENROLLEE,SCORE_SNP_NEW_ENROLLEE
0,P001,1,1950-06-15,1,0,0,75,75,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,P002,2,1980-03-10,0,1,1,45,45,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,P003,1,1965-12-01,0,0,0,60,60,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
scored_df

,MBI,SEX,DOB,LTIMCAID,NEMCAID,OREC,AGEF,AGEF_EDIT,DISABL,ORIGDS,...,MCAID_ORIGDIS_NEM68,MCAID_ORIGDIS_NEM69,MCAID_ORIGDIS_NEM70_74,MCAID_ORIGDIS_NEM75_79,MCAID_ORIGDIS_NEM80_84,MCAID_ORIGDIS_NEM85_89,MCAID_ORIGDIS_NEM90_94,MCAID_ORIGDIS_NEM95_GT,SCORE_NEW_ENROLLEE,SCORE_SNP_NEW_ENROLLEE
0,P001,1,1950-06-15,1,0,0,75,75,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,P002,2,1980-03-10,0,1,1,45,45,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,P003,1,1965-12-01,0,0,0,60,60,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
# 5) Apply KEEP list equivalent to SAS V2825T2P and save CSV [file:38][file:31]

keepvars = [
    ID_COL,
    "SEX", "DOB", "LTIMCAID", "NEMCAID", "OREC",
    *DEM_VARS,
    *HCC_LIST,
    *CC_LIST,
    "SCORE_COMMUNITY_NA",
    "SCORE_COMMUNITY_ND",
    "SCORE_COMMUNITY_FBA",
    "SCORE_COMMUNITY_FBD",
    "SCORE_COMMUNITY_PBA",
    "SCORE_COMMUNITY_PBD",
    "SCORE_INSTITUTIONAL",
    "SCORE_NEW_ENROLLEE",
    "SCORE_SNP_NEW_ENROLLEE",
]

existing_keep = [c for c in keepvars if c in scored_df.columns]
final_df = scored_df[existing_keep].copy()
print("Final shape:", final_df.shape)
final_df.head()


Final shape: (3, 304)


,MBI,SEX,DOB,LTIMCAID,NEMCAID,OREC,AGEF,ORIGDS,DISABL,F0_34,...,CC463,SCORE_COMMUNITY_NA,SCORE_COMMUNITY_ND,SCORE_COMMUNITY_FBA,SCORE_COMMUNITY_FBD,SCORE_COMMUNITY_PBA,SCORE_COMMUNITY_PBD,SCORE_INSTITUTIONAL,SCORE_NEW_ENROLLEE,SCORE_SNP_NEW_ENROLLEE
0,P001,1,1950-06-15,1,0,0,75,0,0,0,...,0,0.502,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,P002,2,1980-03-10,0,1,1,45,0,1,0,...,0,0.000,0.340,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,P003,1,1965-12-01,0,0,0,60,0,0,0,...,0,0.000,0.345,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
def choose_final_score(row):
    # AGEF is the SAS age variable carried through your code
    age = row["AGEF"]
    orec = str(row["OREC"])
    ltimcaid = row["LTIMCAID"]
    nemcaid = row["NEMCAID"]

    # --- Status flags (adjust to your data) ---
    # New Enrollee vs continuing (you may have a dedicated NE flag; NEMCAID is a proxy here)
    is_ne = nemcaid > 0

    # Institutional vs community
    is_inst = ltimcaid == 1

    # Aged vs disabled (CMS convention: originally disabled if OREC='1')
    is_aged = (age >= 65) and (orec != "1")
    is_disabled = (age < 65) or (orec == "1")

    # Dual status flags – plug in your own columns
    is_full_dual = row.get("FULL_DUAL", 0) == 1
    is_partial_dual = row.get("PART_DUAL", 0) == 1
    is_non_dual = not (is_full_dual or is_partial_dual)

    # SNP flag – adjust to your column name
    is_snp = row.get("SNP", 0) == 1

    # --- Score choice logic ---
    if is_ne:
        if is_snp:
            return row["SCORE_SNP_NEW_ENROLLEE"]
        else:
            return row["SCORE_NEW_ENROLLEE"]

    if is_inst:
        return row["SCORE_INSTITUTIONAL"]

    if is_non_dual and is_aged:
        return row["SCORE_COMMUNITY_NA"]
    if is_non_dual and is_disabled:
        return row["SCORE_COMMUNITY_ND"]
    if is_full_dual and is_aged:
        return row["SCORE_COMMUNITY_FBA"]
    if is_full_dual and is_disabled:
        return row["SCORE_COMMUNITY_FBD"]
    if is_partial_dual and is_aged:
        return row["SCORE_COMMUNITY_PBA"]
    if is_partial_dual and is_disabled:
        return row["SCORE_COMMUNITY_PBD"]

    return 0.0

final_df["FINAL_SCORE"] = scored_df.apply(choose_final_score, axis=1)


In [8]:
# 6) Write CSV output
final_df.to_csv(output_path, index=False)
output_path


NameError: name 'final_df' is not defined

In [40]:
output_path = "cms_v28_hcc_scores_with_final.csv"
final_df.to_csv(output_path, index=False)
print("Wrote:", output_path)


Wrote: cms_v28_hcc_scores_with_final.csv


In [41]:
import os
os.getcwd()          # shows the folder
os.listdir()         # should list "cms_v28_hcc_scores_with_final.csv"


['.gitignore',
 '.ipynb_checkpoints',
 'CMS-HCC V28.ipynb',
 'cms_v28_hcc_scores_with_final.csv',
 'data',
 'etc',
 'Include',
 'Lib',
 'pyvenv.cfg',
 'Scripts',
 'share']